# Tensorflow ImageNet-ResNet50 Patch Demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the ImageNet dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

This notebook demonstrates the adversarial patch attack applied on the ResNet50 model.

The following two sections cover experiment setup and is similar across all demos.

## Setup: Experiment Name and ImageNet Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name."
- Ensure that the `DATASET_DIR` parameter is set to the location of the ImageNet dataset directory. Currently set to `/nfs/data/ImageNet-Kaggle-2017"` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/ImageNet-Kaggle-2017"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_adversarial_patch"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Dioptra API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the DIOPTRA_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Path to custom task plugins archives
CUSTOM_PLUGINS_PATCH_TAR_GZ = Path("custom-plugins-patch.tar.gz")

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both `train` and `val-sorted` contain a full set of labeled training and test images.
The `val-sorted-#` directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

Each of the directories listed follows the general subfolder structure below:

    ImageNet
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    ├── val-sorted
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    ...

The subfolders under each directory are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [2]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `DIOPTRA_RESTAPI_URI` to figure out how to connect to the Dioptra RESTful API,

In [3]:
restapi_client = utils.DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2022-05-15T00:54:38.372693',
 'name': 'howard_imagenet_adversarial_patch',
 'experimentId': 3,
 'lastModified': '2022-05-15T00:54:38.372693'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Dioptra API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'createdOn': '2022-05-04T22:01:06.622195',
  'name': 'tensorflow_cpu',
  'queueId': 1,
  'lastModified': '2022-05-04T22:01:06.622195'},
 {'createdOn': '2022-05-04T22:01:07.551981',
  'name': 'tensorflow_gpu',
  'queueId': 2,
  'lastModified': '2022-05-04T22:01:07.551981'},
 {'createdOn': '2022-05-04T22:01:08.473076',
  'name': 'pytorch_cpu',
  'queueId': 3,
  'lastModified': '2022-05-04T22:01:08.473076'},
 {'createdOn': '2022-05-04T22:01:09.418804',
  'name': 'pytorch_gpu',
  'queueId': 4,
  'lastModified': '2022-05-04T22:01:09.418804'}]

This example also makes use of the `custom_patch_plugins` package stored locally under the `task-plugins/dioptra_custom/custom_patch_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [6]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

make: Nothing to be done for 'custom-plugins'.


Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [7]:
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_patch_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_patch_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_PATCH_TAR_GZ,
    )

response_custom_plugins

{'taskPluginName': 'custom_patch_plugins',
 'modules': ['estimators_keras_classifiers.py',
  'import_keras.py',
  'registry_art.py',
  '__init__.py',
  'defenses_image_preprocessing.py',
  'tensorflow.py',
  'data_tensorflow.py',
  'attacks_patch.py'],
 'collection': 'dioptra_custom'}

If at any point you need to update one or more files within the `custom_patch_plugins` package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'custom_patch_plugins' package
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
```

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [8]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response


# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## Adversarial Patches: ImageNet Evaluation

Now, we will test the pretrained ResNet50 model on a subset of the ImageNet dataset.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [9]:
response_resnet50_init = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        f"-P register_model_name={EXPERIMENT_NAME}_pretrained_resnet50",
        "-P imagenet_preprocessing=True",
        "-P image_size=224,224,3",
        "-P model_architecture=resnet50",
        f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Initialization job for ResNet50 neural network submitted")
print("")
pprint.pprint(response_resnet50_init)

response_resnet50_init = get_run_id(response_resnet50_init)

Initialization job for ResNet50 neural network submitted

{'createdOn': '2022-05-20T19:06:52.490501',
 'dependsOn': None,
 'entryPoint': 'init_model',
 'entryPointKwargs': '-P batch_size=20 -P '
                     'register_model_name=howard_imagenet_adversarial_patch_pretrained_resnet50 '
                     '-P imagenet_preprocessing=True -P image_size=224,224,3 '
                     '-P model_architecture=resnet50 -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000',
 'experimentId': 3,
 'jobId': '3dfdb0e1-9acb-4758-9b25-79d1c7235ce0',
 'lastModified': '2022-05-20T19:06:52.490501',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/b20ed754be304c93974388d9d9f38dac/workflows.tar.gz'}


### Deploying and Testing Adversarial Patches

Now we will apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.

In [10]:
# Create Patches
response_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P num_patch_gen_samples=10",
            "-P num_patch=1",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_init["jobId"],
)

print("Patch attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_patch)
print("")

response_resnet50_patch = get_run_id(response_resnet50_patch)

Patch attack (ResNet50 architecture) job submitted

{'createdOn': '2022-05-20T19:07:00.867651',
 'dependsOn': '3dfdb0e1-9acb-4758-9b25-79d1c7235ce0',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P '
                     'model_name=howard_imagenet_adversarial_patch_pretrained_resnet50 '
                     '-P model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P num_patch_gen_samples=10 -P num_patch=1',
 'experimentId': 3,
 'jobId': '0de71551-6009-4a07-97f6-c2ef82ec6b1e',
 'lastModified': '2022-05-20T19:07:00.867651',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/39fd66598b36494d92a12a20d4eaed17/workflows.tar.gz'}



In [11]:
# Deploy Patch attack on test set.
response_deploy_resnet50_patch_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_resnet50_patch['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            f"-P data_dir={DATASET_DIR}/images/ILSVRC/Data/CLS-LOC/val-sorted-5000",
            "-P batch_size=500",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_resnet50_patch["jobId"],
)

print("Patch deployment (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_deploy_resnet50_patch_testing)
print("")

response_deploy_resnet50_patch_testing = get_run_id(response_deploy_resnet50_patch_testing)

Patch deployment (ResNet50 architecture) job submitted

{'createdOn': '2022-05-20T19:08:17.218459',
 'dependsOn': '0de71551-6009-4a07-97f6-c2ef82ec6b1e',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=3db65c34315641749cbe46e5ea3e9287 -P '
                     'model_name=howard_imagenet_adversarial_patch_pretrained_resnet50 '
                     '-P model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-5000 '
                     '-P batch_size=500',
 'experimentId': 3,
 'jobId': '1993d04d-906c-4903-97d4-1fd264686bcf',
 'lastModified': '2022-05-20T19:08:17.218459',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/87543199a34245a7906d44aca262285e/workflows.tar.gz'}



In [12]:
# Check patched dataset results   
response_resnet50_infer_resnet50_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_resnet50_patch_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            f"-P model_version=none",
            "-P image_size=224,224,3",
            "-P imagenet_preprocessing=True",
            "-P batch_size=100",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_resnet50_patch_testing["jobId"],
)

print("Patch evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_resnet50_infer_resnet50_patch)
print("")

response_resnet50_infer_resnet50_patch = get_run_id(response_resnet50_infer_resnet50_patch)
print_mlflow_results(response_resnet50_infer_resnet50_patch)

Patch evaluation (ResNet50 architecture) job submitted

{'createdOn': '2022-05-20T19:18:07.489069',
 'dependsOn': '1993d04d-906c-4903-97d4-1fd264686bcf',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=157f123fb1354ca09dcb76f78af95091 -P '
                     'model_name=howard_imagenet_adversarial_patch_pretrained_resnet50 '
                     '-P model_version=none -P image_size=224,224,3 -P '
                     'imagenet_preprocessing=True -P batch_size=100 -P '
                     'adv_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'adv_data_dir=adv_patch_dataset',
 'experimentId': 3,
 'jobId': 'c0e5b0f3-5b39-4ee8-be3a-96d8c466f1c7',
 'lastModified': '2022-05-20T19:18:07.489069',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/9011c4dc91e74408be358575411e8313/workflows.tar.gz'}

{'accuracy': 0.5784000158309937,
 'auc': 0.9367275238037109,
 'loss': 1.9498850357532502,
 'precision': 0.74146

## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [13]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the ResNet50 architecture,

In [14]:
run_resnet50 = mlflow_client.get_run(response_resnet50_infer_resnet50_patch["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [15]:
pprint.pprint(run_resnet50.data.metrics)

{'accuracy': 0.5784000158309937,
 'auc': 0.9367275238037109,
 'loss': 1.9498850357532502,
 'precision': 0.7414655685424805,
 'recall': 0.5038999915122986}


To review the run's parameters, we use,

In [16]:
pprint.pprint(run_resnet50.data.params)

{'adv_data_dir': 'adv_patch_dataset',
 'adv_tar_name': 'adversarial_patch_dataset.tar.gz',
 'batch_size': '100',
 'dataset_seed': '1986865553',
 'entry_point_seed': '56718301370532184666262495574641970180',
 'image_size': '224,224,3',
 'imagenet_preprocessing': 'True',
 'model_name': 'howard_imagenet_adversarial_patch_pretrained_resnet50',
 'model_version': 'none',
 'run_id': '157f123fb1354ca09dcb76f78af95091',
 'seed': '-1',
 'tensorflow_global_seed': '863168140'}


To review the run's tags, we use,

In [17]:
pprint.pprint(run_resnet50.data.tags)

{'dioptra.dependsOn': '1993d04d-906c-4903-97d4-1fd264686bcf',
 'dioptra.jobId': 'c0e5b0f3-5b39-4ee8-be3a-96d8c466f1c7',
 'dioptra.queue': 'tensorflow_gpu',
 'mlflow.project.backend': 'dioptra',
 'mlflow.project.entryPoint': 'infer',
 'mlflow.source.name': '/work/tmpnygso0fx',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'dioptra'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).